# Regresión con KNN

In [7]:
import numpy as np
import pandas as pd
# Entorno Scikit Learn
from sklearn.neighbors import KNeighborsRegressor   # Algoritmo
from sklearn.metrics import mean_squared_error, mean_absolute_error      # Metrica de desempeño ( para ver que tan bien se ajusta nuestro modelo a los datos)
from sklearn.model_selection import train_test_split    # Para dividir el conjunto de prueba y entrenamiento
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score   # Validación cruzada 
from sklearn.model_selection import GridSearchCV
# Para gráficos
import matplotlib.pyplot as plt


In [8]:
# Datos
data=pd.read_csv("https://raw.githubusercontent.com/Albertuff/Machine-Learning/master/datos/estatura_peso.csv")
data.head()

,gender,height,weight
0,0,63.2,168.7
1,0,68.7,169.8
2,0,64.8,176.6
3,0,67.9,246.8
4,1,68.9,151.6


In [9]:
# Preprocesamiento

#   Definimos los atributos y la variable de respuesta
X=data[["height"]]        # Atributo: Estatura
Y=data["weight"]        # Target o Respuesta: Peso   --  El target debe ser una variable continua cuando se usa KNN para regresión

# Definimos el conjunto de entrenamiento y prueba
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2,random_state=1234)

# Definimos el modelo
Modelo=KNeighborsRegressor(n_neighbors=7,n_jobs=-1)

# Entrenamos el modelo
Modelo.fit(X_train,Y_train)

# Realizamos las predicciones
Y_pred=Modelo.predict(X_test)
#print(Y_pred)
#print(Y_test)

# Que calificación le asignamos a nuestro modelo 
print(f" El error absoluto promedio es: {mean_absolute_error(Y_test,Y_pred):.4f} lbs")
print(f" El error cuadratico promedio es: {mean_squared_error(Y_test,Y_pred):.4f} lbs^2")
print(f" La raíz cuadrada del error cuadratico promedio es: {mean_squared_error(Y_test,Y_pred,squared=False):.4f} lbs") # squared=False regresa la raiz del error cuadratico promedio. 


 El error absoluto promedio es: 32.3255 lbs
 El error cuadratico promedio es: 1699.5376 lbs^2
 La raíz cuadrada del error cuadratico promedio es: 41.2254 lbs


 ## Los resultados dependen de varios factores:
 
 1.- Las variables predictoras X, ¿Son adecuadas, ¿suficientes?, ¿unidades de medición?

 2.- Los resultados dependen del conjunto de entrenamiento y prueba
 
 3.- Los resultados dependen del valor del hiperparámetro k


In [10]:
# Realizamos el preprocesamiento de las covariables o atributos
# Vamos a hacer la tipificaxión de la variable estatura
from sklearn import preprocessing
z_score=preprocessing.StandardScaler()
X=z_score.fit_transform(X)

# Definimos el conjunto de entrenamiento y prueba
X_train, X_test, Y_train, Y_test=train_test_split(X,Y,test_size=0.2,random_state=1234)

# Definimos el modelo
Modelo=KNeighborsRegressor(n_neighbors=7,n_jobs=-1)

# Entrenamos el modelo
Modelo.fit(X_train,Y_train)

# Realizamos las predicciones
Y_pred=Modelo.predict(X_test)
#print(Y_pred)
#print(Y_test)

# Que calificación le asignamos a nuestro modelo 
print(f" El error absoluto promedio es: {mean_absolute_error(Y_test,Y_pred):.4f} lbs")
print(f" El error cuadratico promedio es: {mean_squared_error(Y_test,Y_pred):.4f} lbs^2")
print(f" La raíz cuadrada del error cuadratico promedio es: {mean_squared_error(Y_test,Y_pred,squared=False):.4f} lbs") # squared=False regresa la raiz del error cuadratico promedio

 El error absoluto promedio es: 32.2051 lbs
 El error cuadratico promedio es: 1690.6996 lbs^2
 La raíz cuadrada del error cuadratico promedio es: 41.1181 lbs


In [11]:
# Calcular una metrica y calibrar el modelo tiene el error de que el conjunto de prueba y entrenamiento están relacionados
#  porque se estandarizaron utilizando los valores globales de la media y la varianza

# Primero hacemos la división del conjunto de entrenamiento y prueba
train, test=train_test_split(data,test_size=0.2,random_state=1234)

# Definimos los atributos y la variable de respuesta
X_train=train[["height"]]
Y_train=train["weight"]
X_test=test[["height"]]
Y_test=test["weight"]

# Vamos a hacer la tipificaxión de la variable estatura
z_score=preprocessing.StandardScaler()
X_train=z_score.fit_transform(X_train)


# Definimos el modelo
Modelo=KNeighborsRegressor(n_neighbors=7,n_jobs=-1)

# Entrenamos el modelo
Modelo.fit(X_train,Y_train)

# Realizamos las predicciones, primero tipificando los datos
X_test=z_score.fit_transform(X_test)
Y_pred=Modelo.predict(X_test)
#print(Y_pred)
#print(Y_test)

# Que calificación le asignamos a nuestro modelo 
print(f" El error absoluto promedio es: {mean_absolute_error(Y_test,Y_pred):.4f} lbs")
print(f" El error cuadratico promedio es: {mean_squared_error(Y_test,Y_pred):.4f} lbs^2")
print(f" La raíz cuadrada del error cuadratico promedio es: {mean_squared_error(Y_test,Y_pred,squared=False):.4f} lbs") # squared=False regresa la raiz del error cuadratico promedio

 El error absoluto promedio es: 33.9653 lbs
 El error cuadratico promedio es: 1805.0755 lbs^2
 La raíz cuadrada del error cuadratico promedio es: 42.4862 lbs


In [17]:
# Buscamos el valor de k con el cual obtenemos el mejor desempeño
# Definimos los atributos y la variable objertivo
X=data[["height"]]
Y=data["weight"]

parametros={"n_neighbors":np.arange(1,50)}   # Buscamos el mejor valor de k, el numero de vecinos, entre 1 - 49
                                             # El espacio de parametros está definido como un diccionario

# El modelo que vamos a entrenar es el de k vecinos mas cercanos
modelo=KNeighborsRegressor()

# Definimos el tipo de busqueda
# Primero definimos la rejilla
cv=KFold(n_splits=10)
rejilla=GridSearchCV(modelo,param_grid=parametros,scoring="neg_mean_absolute_error",cv=cv,n_jobs=-1)
rejilla.fit(X,Y)

print(f" La mejor configuración de k es: {rejilla.best_params_}")
print(f" El error absoluto promedio es : {mean_absolute_error(Y,rejilla.best_estimator_.predict(X)):.8f}")

# Hasta aquí ya tenemos el mejor modelo que es con 7 vecinos
# Cual es el desempeño predictivo del modelo con 7 vecinos?, Lo podemos saber a travez de una validacion cruzada anidada

 La mejor configuración de k es: {'n_neighbors': 7}
 El error absoluto promedio es : 23.71265306


In [13]:
# Realizamos una validación cruzada anidada

# Definimos el esquema de validación cruzada interna. Que selecciona el mejor modelo
cv_interna=KFold(n_splits=10)

# Definimos la validacion cruzada externa, la cual permite calificar el desempeño predictivo del mejor modelo seleccionado
cv_externa=KFold(n_splits=10)

# El modelo a entrenar es k vecinos mas cercanos
Modelofinal=KNeighborsRegressor()

# Realizamos la busqueda de la mejor configuración
rejilla=GridSearchCV(Modelofinal,param_grid=parametros,scoring="neg_mean_absolute_error",n_jobs=-1,cv=cv_interna)

# Ya que encontro el mejor modelo, lo entrena en el conjunto de entrenamiento y lo prueba en el conjunto de prueba
scores=cross_val_score(rejilla,X,Y,cv=cv_externa,scoring="neg_mean_absolute_error",n_jobs=-1)

print(f" El desempeño predictivo promedio del mejor modelo es: {-scores.mean():.4f}")

 El desempeño predictivo promedio del mejor modelo es: 27.0405


## Clase 12 
# Pipeline: O secuencia de funciones de preprocesamiento y entrenamiento

In [30]:
from sklearn.pipeline import Pipeline


# Primero se arreglan los datos faltantes si es que hay, luego se hace el reescalamiento y despues se agrega el modelo
Pipe=Pipeline([("zscore",preprocessing.StandardScaler()),("Modelo",KNeighborsRegressor())]) #Los preocesos se definen como una lista
# El hiperparámetro del modelo es: n_neighbors

# Definimos el método de busqueda
espacio_parametros={"Modelo__n_neighbors":np.arange(1,50)}  # Para pasar los hiperparámetros al pipe es necesario, anteponer al nomobre del hiperparámetro
                                                            # el nombre que le estamos dando en el pipe a nuestro algoritmo, enn este caso: Modelo, seguido de dos guiones bajos
                                                            # Modelo__

rejilla=GridSearchCV(Pipe,param_grid=espacio_parametros,scoring="neg_mean_absolute_error",n_jobs=-1,cv=10)
rejilla.fit(X,Y)
print(f" La mejor configuración del hiperparámetro k es: {rejilla.best_params_}")
print(f" El mejor score (error absoluto promedio) es: {-rejilla.best_score_}")


 La mejor configuración del hiperparámetro k es: {'Modelo__n_neighbors': 7}
 El mejor score (error absoluto promedio) es: 25.392857142857146


In [32]:
# Sabemos que este valor del score, es una metrica "optimista". Para tener uan metrica mas realista, hacemos una validacion cruzada anidada

# cv_interna=KFold(n_splits=10)
# cv_externa=KFold(n_splits=10)

rejilla=GridSearchCV(Pipe,param_grid=espacio_parametros,scoring="neg_mean_absolute_error",n_jobs=-1,cv=10)
puntajes=cross_val_score(rejilla,X,Y,cv=10,scoring="neg_mean_absolute_error",n_jobs=-1)
print(f" El mejor score (error absoluto promedio) es: {-puntajes.mean()}")

 El mejor score (error absoluto promedio) es: 27.241248150676103


In [33]:
# Otro tipo de validación cruzada

cv_interna=KFold(n_splits=10)
cv_externa=RepeatedKFold(n_splits=10,n_repeats=5)
rejilla=GridSearchCV(Pipe,param_grid=espacio_parametros,scoring="neg_mean_absolute_error",n_jobs=-1,cv=cv_interna)
puntajes=cross_val_score(rejilla,X,Y,cv=cv_externa,scoring="neg_mean_absolute_error",n_jobs=-1)
print(f" El mejor score (error absoluto promedio) es: {-puntajes.mean()}")

 El mejor score (error absoluto promedio) es: 27.50561641337778
